# Interactive Widget: Front End Code

Throughout this workbook, we used steps from the following web pages to inform our widgets.
- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html
- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
- https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

### Set Up

In [4]:
# Import the necessary data libraries.
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression

# The following are for Jupyter Widgets.
import ipywidgets as widgets
from IPython.display import display
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

In [2]:
# Set up datasets.
X_resampled_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/interim/X_resampled_forWidget.csv'
X_resampled = pd.read_csv(X_resampled_url, sep = ',', engine = 'python')
y_resampled_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/interim/y_resampled_forWidget.csv'
y_resampled = pd.read_csv(y_resampled_url, sep = ',', engine = 'python')

We know from testing the type of `y_resampled` in `InteractiveWidget_BackEnd.ipynb` that `y_resampled` needs to be a series in order for our model to run correctly. We also know from this site (https://datatofish.com/pandas-dataframe-to-series/) how to change a dataframe into a series.

In [3]:
# Change the y_resampled dataframe into a y_resampled series.
y_resampled = y_resampled.squeeze()

### Create the Modeling Function

In [5]:
# Create the function widgetpred. We'll use this in the function predict.
def widgetpred(X_resampled, y_resampled, X_test, estimator, **kwargs):
    """
    Test various estimators.
    """
    # Instantiate the classification model and visualizer
    estimator.fit(X_resampled, y_resampled, **kwargs)  
    
    predicted = estimator.predict(X_test)
    
    # Compute and return F1 (harmonic mean of precision and recall)
    return predicted

### Create the Widget

In [6]:
"""
Establish function "predict" which allows selection of three nationalities,
countries, and circuit tiers, as well as input one of each of the following values:
grid, alt, average_lap_time, minimum_lap_time, PRCP, TAVG, TMAX, TMIN.

Place these values in the dataframe input_df and display the dataframe.

Create prediction based on widgetpred function and display the prediction:
0 for did not finish, 1 for did finish.
"""
def predictfinish(nationality, country, circuit, grid, alt, average_lap_time, minimum_lap_time, PRCP, TAVG, TMAX, TMIN):
    # Use an elif statement to determine the output one-hot encoding based on the input nationality.
    if nationality == "German":
        nationality_CompletionStatus_1 = 0.209566
        nationality_CompletionStatus_2 = 0.790434
    elif nationality == "British":
        nationality_CompletionStatus_1 = 0.240838
        nationality_CompletionStatus_2 = 0.759162
    else:
        nationality_CompletionStatus_1 = 0.292359
        nationality_CompletionStatus_2 = 0.707641
    
    # Use an elif statement to determine the output one-hot encoding based on the input country.
    if country == "Italy":
        country_CompletionStatus_1 = 0.279099
        country_CompletionStatus_2 = 0.720901
    elif country == "Germany":
        country_CompletionStatus_1 = 0.291429
        country_CompletionStatus_2 = 0.708571
    else:
        country_CompletionStatus_1 = 0.219697
        country_CompletionStatus_2 = 0.780303
    
    # Use an elif statement to determine the output one-hot encoding based on the input circuit.
    if circuit == "Tier1":
        binned_circuits_CompletionStatus_1 = 0.253451
        binned_circuits_CompletionStatus_2 = 0.746549
    elif circuit == "Tier2":
        binned_circuits_CompletionStatus_1 = 0.277588
        binned_circuits_CompletionStatus_2 = 0.722412
    else:
        binned_circuits_CompletionStatus_1 = 0.235686
        binned_circuits_CompletionStatus_2 = 0.764314
    
    # Establish the data of our input_df dataframe.
    inputdata = [[nationality_CompletionStatus_1, nationality_CompletionStatus_2,
                country_CompletionStatus_1, country_CompletionStatus_2,
                binned_circuits_CompletionStatus_1, binned_circuits_CompletionStatus_2,
                grid, alt, average_lap_time, minimum_lap_time, PRCP, TAVG, TMAX, TMIN]]
    
    # Establish the dataframe input_df itself with pd.DataFrame.
    input_df = pd.DataFrame(inputdata, columns =
                ["nationality_CompletionStatus_1", "nationality_CompletionStatus_2",
                "country_CompletionStatus_1", "country_CompletionStatus_2",
                "binned_circuits_CompletionStatus_1", "binned_circuits_CompletionStatus_2",
                "grid", "alt", "average_lap_time", "minimum_lap_time", "PRCP", "TAVG", "TMAX", "TMIN"])
    
    display(input_df)
    
    # Using the widgetpred function, predict whether the car will finish the race or not given input_df.
    pred = widgetpred(X_resampled, y_resampled, input_df, LogisticRegression(solver='lbfgs'))
    
    # Using an if-else statement, determine what interactors will see given the data they input.
    if pred[0] == 1:
        writtenpred = "finish the race."
    else:
        writtenpred = "not finish the race."
    
    print("According to our Logistic Regression model, your car is predicted to", writtenpred)

# Create a widget that will interact with the predictfinish function.
interact(predictfinish, nationality = widgets.Dropdown(options = ["German", "British", "Brazilian"], value = "German"),
         country = widgets.Dropdown(options = ["Italy", "Germany", "Spain"], value = "Italy"),
         circuit = widgets.Dropdown(options = ["Tier1", "Tier2", "Tier3"], value = "Tier1"),
         grid = widgets.BoundedIntText(min = 0, max = 30, description = 'Grid:', disabled = False, continuous_update = False),
         alt = widgets.BoundedFloatText(min = -100, max = 2500, description = 'Altitude:', disabled = False, continuous_update = False),
         average_lap_time = widgets.BoundedFloatText(min = 0, max = 300000, description = 'Average Lap Time:', disabled = False, continuous_update = False),
         minimum_lap_time = widgets.BoundedFloatText(min = 0, max = 300000, description = 'Minimum Lap Time:', disabled = False, continuous_update = False),
         PRCP = widgets.BoundedFloatText(min = 0, max = 20, description = 'Precipitation:', disabled = False, continuous_update = False),
         TAVG = widgets.BoundedFloatText(min = 0, max = 120, description = 'Average Temperature - Fahrenheit:', disabled = False, continuous_update = False),
         TMAX = widgets.BoundedFloatText(min = 0, max = 120, description = 'Max Temperature - Fahrenheit:', disabled = False, continuous_update = False),
         TMIN = widgets.BoundedFloatText(min = 0, max = 120, description = 'Min Temperature - Fahrenheit:', disabled = False, continuous_update = False));

interactive(children=(Dropdown(description='nationality', options=('German', 'British', 'Brazilian'), value='G…